In [1]:
import pandas as pd
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, expr

spark = SparkSession.builder.appName("VolcanicEruptions").getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [2]:
historical_df = pd.read_excel(
    "./datasets/volcano-eruptions-all-historical-to-2020/volcanic_dataset.xls", header=1
)
historical_df.to_csv(
    "./datasets/volcano-eruptions-all-historical-to-2020/volcanic_dataset.csv"
)

,Volcano Number,Volcano Name,Eruption Number,Eruption Category,Area of Activity,VEI,VEI Modifier,Start Year Modifier,Start Year,Start Year Uncertainty,...,Evidence Method (dating),End Year Modifier,End Year,End Year Uncertainty,End Month,End Day Modifier,End Day,End Day Uncertainty,Latitude,Longitude
0,233020,"Fournaise, Piton de la",22343,Confirmed Eruption,NaN,NaN,NaN,NaN,2020.0,NaN,...,Historical Observations,NaN,2020.0,NaN,2.0,NaN,16.0,NaN,-21.244,55.708
1,345020,Rincon de la Vieja,22346,Confirmed Eruption,NaN,NaN,NaN,NaN,2020.0,NaN,...,Historical Observations,NaN,2020.0,NaN,1.0,NaN,31.0,NaN,10.830,-85.324
2,353010,Fernandina,22347,Confirmed Eruption,NaN,NaN,NaN,NaN,2020.0,NaN,...,Historical Observations,NaN,2020.0,NaN,1.0,NaN,12.0,NaN,-0.370,-91.550
3,273070,Taal,22344,Confirmed Eruption,NaN,NaN,NaN,NaN,2020.0,NaN,...,Historical Observations,NaN,2020.0,NaN,1.0,NaN,22.0,NaN,14.002,120.993
4,282050,Kuchinoerabujima,22345,Confirmed Eruption,NaN,NaN,NaN,NaN,2020.0,NaN,...,Historical Observations,>,2020.0,NaN,2.0,NaN,13.0,NaN,30.443,130.217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11167,327812,Red Hill,22193,Confirmed Eruption,Cerro Pomo?,NaN,NaN,NaN,-9850.0,500.0,...,Surface Exposure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.250,-108.830
11168,213020,Nemrut Dagi,13908,Confirmed Eruption,NaN,NaN,NaN,NaN,-9950.0,150.0,...,Varve Count,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.654,42.229
11169,324020,Craters of the Moon,21101,Confirmed Eruption,Sunset cone,0.0,NaN,NaN,-10060.0,NaN,...,Radiocarbon (uncorrected),NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.420,-113.500
11170,222161,Igwisi Hills,22141,Confirmed Eruption,NE Volcano,1.0,NaN,NaN,-10450.0,4800.0,...,Surface Exposure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.889,31.933


In [3]:
dataset_path = (
    "./datasets/volcano-eruptions-all-historical-to-2020/volcanic_dataset.csv"
)


df = (
    spark.read.format("csv")
    .option("delimiter", ",")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(dataset_path)
)


display(df)

_c0,Volcano Number,Volcano Name,Eruption Number,Eruption Category,Area of Activity,VEI,VEI Modifier,Start Year Modifier,Start Year,Start Year Uncertainty,Start Month,Start Day Modifier,Start Day,Start Day Uncertainty,Evidence Method (dating),End Year Modifier,End Year,End Year Uncertainty,End Month,End Day Modifier,End Day,End Day Uncertainty,Latitude,Longitude
0,233020,"Fournaise, Piton ...",22343,Confirmed Eruption,null,null,null,null,2020.0,null,2.0,null,10.0,null,Historical Observ...,null,2020.0,null,2.0,null,16.0,null,-21.244,55.708
1,345020,Rincon de la Vieja,22346,Confirmed Eruption,null,null,null,null,2020.0,null,1.0,null,31.0,null,Historical Observ...,null,2020.0,null,1.0,null,31.0,null,10.83,-85.324
2,353010,Fernandina,22347,Confirmed Eruption,null,null,null,null,2020.0,null,1.0,null,12.0,null,Historical Observ...,null,2020.0,null,1.0,null,12.0,null,-0.37,-91.55
3,273070,Taal,22344,Confirmed Eruption,null,null,null,null,2020.0,null,1.0,null,12.0,null,Historical Observ...,null,2020.0,null,1.0,null,22.0,null,14.002,120.993
4,282050,Kuchinoerabujima,22345,Confirmed Eruption,null,null,null,null,2020.0,null,1.0,null,11.0,null,Historical Observ...,>,2020.0,null,2.0,null,13.0,null,30.443,130.217
5,241040,White Island,22338,Confirmed Eruption,1978/90 Crater Co...,2.0,null,null,2019.0,null,12.0,null,9.0,null,Historical Observ...,null,2019.0,null,12.0,null,9.0,null,-37.52,177.18
6,311060,Semisopochnoi,22341,Confirmed Eruption,null,null,null,null,2019.0,null,12.0,null,7.0,null,Historical Observ...,>,2020.0,null,2.0,null,16.0,null,51.93,179.58
7,284096,Nishinoshima,22340,Confirmed Eruption,null,1.0,null,null,2019.0,null,12.0,null,5.0,null,Historical Observ...,>,2020.0,null,2.0,null,15.0,null,27.247,140.874
8,311240,Cleveland,22342,Uncertain Eruption,null,null,null,null,2019.0,null,11.0,null,7.0,null,Historical Observ...,null,2019.0,null,12.0,?,7.0,null,52.825,-169.944
9,282060,Kikai,22339,Confirmed Eruption,null,null,null,null,2019.0,null,11.0,null,2.0,null,Historical Observ...,>,2020.0,null,2.0,null,11.0,null,30.793,130.305


In [9]:
df.describe()

summary,_c0,Volcano Number,Volcano Name,Eruption Number,Eruption Category,Area of Activity,VEI,VEI Modifier,Start Year Modifier,Start Year,Start Year Uncertainty,Start Month,Start Day Modifier,Start Day,Start Day Uncertainty,Evidence Method (dating),End Year Modifier,End Year,End Year Uncertainty,End Month,End Day Modifier,End Day,End Day Uncertainty,Latitude,Longitude
count,11172,11172,11172,11172,11172,4693,8267,1312,1612,11170,2089,10982,416,10979,854,9892,102,4330,6,4327,718,4324,693,11171,11172
mean,5585.5,300263.2551915503,null,15663.315252416756,null,null,1.9477377207839341,0.0,null,624.6581915846016,293.18956438487317,3.4504643962848296,9.0,7.013389197558976,64.45901639344262,null,null,1917.2789838337183,3.3333333333333335,6.202449734226947,null,13.329787234042554,23.881673881673883,16.891894369349316,31.625019244540226
stddev,3225.222938030796,52321.6140133988,null,3294.855563983139,null,null,1.160836145200808,null,null,2477.9648275070135,662.2651332875982,4.071626180453439,null,9.646394503931674,97.75770982732959,null,null,157.66721900595363,3.614784456460256,3.7055472539712775,null,9.842851775139835,43.06454856211558,30.749283806982838,115.23814266940296
min,0,210010,Abu,10001,Confirmed Eruption,"""""""Brimstone Island",""""" W of Macauley ...",+,<,-10450.0,1.0,0.0,9.0,0.0,1.0,Anthropology,<,-475.0,1.0,0.0,<,0.0,1.0,-77.53,-179.97
max,11171,600000,Zuni-Bandera,22348,Uncertain Eruption,Öskjuvatn Caldera...,7.0,^,?,2020.0,14000.0,12.0,?,31.0,730.0,Varve Count,?,2020.0,10.0,12.0,?,31.0,365.0,85.608,179.58
